### Selected Word Group In Youtube Sentence

In [98]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [99]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected Remain Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_Remain_Word"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = True  # True, False
sent_time_start_time = 700  # sn
random_sample = True  # True, False

In [100]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [101]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [102]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [103]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [104]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [105]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [106]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [107]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [108]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [110]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,549.560,553.080,böyle yani yağ dökmüşsün gibi bebek yağı gibi ...,L3bf7dlR5hk
1,20.120,23.860,heder etti keyfekeder,O9HN_8FT3T0
2,76.190,79.693,o kadar üzgünüm ki,A5J0zkFt30c
3,558.334,559.175,bilmiyorum,im2J3GRCnGk
4,35.660,37.620,yanlış kozmetik,hzuaVN5U5oM
...,...,...,...,...
1050454,0.000,4.218,yeni yol arkadaşım ece ile tanıştırmak istiyor...,ZprTu2wzM_I
1050455,567.950,568.990,ya,XqR12aq1uBc
1050456,171.312,175.287,çok çok opak hemen rengini anında veren sürümü...,OU5trL8SmeA
1050457,31.779,33.478,nehir ne oteli,GB8zfRtU7kQ


In [111]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,adam yine geldi
1,al bakalım dostum
2,al şunu al
3,al şunu da
4,ama asla olmaz
...,...
222,yoksa bizi mi
223,yoksa bugün mü
224,yoksa sizi de
225,zaman bugün mü


In [ ]:
# Option
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [112]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [113]:
len(search_list)

227

In [114]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama hadi,330.780,334.680,i stanbullu kafan yarıldı hadi ama hadi siz ar...,Jz5D4A_mDxo
1,hadi ama hadi,594.780,598.220,i stanbullu kafan yarıldı hadi ama hadi siz ar...,N4YDaU51lvI
2,bütün gün burada,16.000,17.976,bütün gün burada kalmayı,ECxkyosOdB4
3,bütün gün burada,690.256,694.335,ne yapacaksın ha bütün gün burada mı tutacaksı...,ioHd1Rcp5Ew
4,bütün gün burada,2.787,5.891,beni bütün gün burada yalnız bırakmanın sebebi...,BVBG_NetyKM
...,...,...,...,...,...
175,al şunu da,642.889,644.749,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
176,al şunu da,657.980,659.080,haydi al şunu da,OcZJIp3KUYs
177,neler oluyor burada,147.700,150.176,eda neler oluyor burada,bsQD5xMYc7o
178,neler oluyor burada,651.420,652.620,neler oluyor burada,es2HApj6HwY


In [115]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama hadi,332.521071,333.565714,i stanbullu kafan yarıldı hadi ama hadi siz ar...,Jz5D4A_mDxo
1,hadi ama hadi,596.315714,597.237143,i stanbullu kafan yarıldı hadi ama hadi siz ar...,N4YDaU51lvI
2,bütün gün burada,16.000000,17.399667,bütün gün burada kalmayı,ECxkyosOdB4
3,bütün gün burada,691.511077,692.923038,ne yapacaksın ha bütün gün burada mı tutacaksı...,ioHd1Rcp5Ew
4,bütün gün burada,2.957082,3.722452,beni bütün gün burada yalnız bırakmanın sebebi...,BVBG_NetyKM
...,...,...,...,...,...
175,al şunu da,643.789000,644.509000,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
176,al şunu da,658.323750,659.080000,haydi al şunu da,OcZJIp3KUYs
177,neler oluyor burada,148.022957,150.176000,eda neler oluyor burada,bsQD5xMYc7o
178,neler oluyor burada,651.420000,652.620000,neler oluyor burada,es2HApj6HwY


In [116]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi ama hadi,331.921071,334.165714,i stanbullu kafan yarıldı hadi ama hadi siz ar...,Jz5D4A_mDxo
1,hadi ama hadi,595.715714,597.837143,i stanbullu kafan yarıldı hadi ama hadi siz ar...,N4YDaU51lvI
2,bütün gün burada,15.400000,17.999667,bütün gün burada kalmayı,ECxkyosOdB4
3,bütün gün burada,690.911077,693.523038,ne yapacaksın ha bütün gün burada mı tutacaksı...,ioHd1Rcp5Ew
4,bütün gün burada,2.357082,4.322452,beni bütün gün burada yalnız bırakmanın sebebi...,BVBG_NetyKM
...,...,...,...,...,...
175,al şunu da,643.189000,645.109000,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
176,al şunu da,657.723750,659.680000,haydi al şunu da,OcZJIp3KUYs
177,neler oluyor burada,147.422957,150.776000,eda neler oluyor burada,bsQD5xMYc7o
178,neler oluyor burada,650.820000,653.220000,neler oluyor burada,es2HApj6HwY


In [117]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,hadi ama hadi,332,334,i stanbullu kafan yarıldı hadi ama hadi siz ar...,Jz5D4A_mDxo
1,hadi ama hadi,596,598,i stanbullu kafan yarıldı hadi ama hadi siz ar...,N4YDaU51lvI
2,bütün gün burada,15,18,bütün gün burada kalmayı,ECxkyosOdB4
3,bütün gün burada,691,694,ne yapacaksın ha bütün gün burada mı tutacaksı...,ioHd1Rcp5Ew
4,bütün gün burada,2,4,beni bütün gün burada yalnız bırakmanın sebebi...,BVBG_NetyKM
...,...,...,...,...,...
175,al şunu da,643,645,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
176,al şunu da,658,660,haydi al şunu da,OcZJIp3KUYs
177,neler oluyor burada,147,151,eda neler oluyor burada,bsQD5xMYc7o
178,neler oluyor burada,651,653,neler oluyor burada,es2HApj6HwY


In [118]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hadi ama hadi,332,334,i stanbullu kafan yarıldı hadi ama hadi siz ar...,Jz5D4A_mDxo,https://www.youtube.com/watch?v=Jz5D4A_mDxo&t=...
1,hadi ama hadi,596,598,i stanbullu kafan yarıldı hadi ama hadi siz ar...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
2,bütün gün burada,15,18,bütün gün burada kalmayı,ECxkyosOdB4,https://www.youtube.com/watch?v=ECxkyosOdB4&t=15s
3,bütün gün burada,691,694,ne yapacaksın ha bütün gün burada mı tutacaksı...,ioHd1Rcp5Ew,https://www.youtube.com/watch?v=ioHd1Rcp5Ew&t=...
4,bütün gün burada,2,4,beni bütün gün burada yalnız bırakmanın sebebi...,BVBG_NetyKM,https://www.youtube.com/watch?v=BVBG_NetyKM&t=2s
...,...,...,...,...,...,...
175,al şunu da,643,645,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg,https://www.youtube.com/watch?v=5e5vf1WrQYg&t=...
176,al şunu da,658,660,haydi al şunu da,OcZJIp3KUYs,https://www.youtube.com/watch?v=OcZJIp3KUYs&t=...
177,neler oluyor burada,147,151,eda neler oluyor burada,bsQD5xMYc7o,https://www.youtube.com/watch?v=bsQD5xMYc7o&t=...
178,neler oluyor burada,651,653,neler oluyor burada,es2HApj6HwY,https://www.youtube.com/watch?v=es2HApj6HwY&t=...


In [119]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

54

In [120]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [121]:
search_all_set.difference(search_result_set)

{'adam yine geldi',
 'al bakalım dostum',
 'ama asla olmaz',
 'ama biliyorum işte',
 'ama zaten biliyorsun',
 'anne orada mısın',
 'artık hiçbir zaman',
 'asla olmaz demek',
 'aslında gerçek değil',
 'aslında gerçek şu',
 'bak gerçekten üzgünüm',
 'bana izin ver',
 'bana ver şunu',
 'ben bile bilmiyorum',
 'ben bile değilim',
 'ben gerçekten üzgünüm',
 'bence bu harika',
 'biliyorsun işte sen',
 'biliyorum işte bu',
 'biliyorum işte o',
 'biliyorum işte tamam',
 'bir küçük kız',
 'biraz izin ver',
 'biri olabilir diye',
 'birlikte olacak mısın',
 'bize doğru geliyor',
 'bu asla olmaz',
 'bu bize lazım',
 'bu fazla uzun',
 'bu harika değil',
 'bu harika olacak',
 'bu küçük adam',
 'bu küçük kız',
 'bu sorun değil',
 'bu yeni geldi',
 'bugün mü oldu',
 'buna izin var',
 'bunu bile bilmiyorum',
 'bunu biliyorum işte',
 'bunu nasıl biliyorsun',
 'bunu nasıl yapıyorsun',
 'bunu tekrar söyle',
 'bunu zaten biliyorsun',
 'bunun için teşekkürler',
 'bunun içinde mi',
 'burada benimle birlikte'

In [122]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,şunu,5
2,tamam,5
3,ver,5
1,yardım,5
4,ama,4
...,...,...
59,daha,1
60,aynı,1
61,bak,1
62,benim,1


In [123]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [124]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [125]:
set_word.difference(set_count)

{'anne',
 'artık',
 'asla',
 'aslında',
 'baba',
 'belki',
 'ben',
 'bence',
 'beni',
 'biliyor',
 'biliyorsun',
 'birkaç',
 'biz',
 'bizi',
 'bizim',
 'bugün',
 'buna',
 'bunun',
 'büyük',
 'de',
 'demek',
 'devam',
 'diye',
 'dostum',
 'doğru',
 'dur',
 'ederim',
 'en',
 'evet',
 'fazla',
 'geldi',
 'geri',
 'gerçek',
 'gerçekten',
 'git',
 'güzel',
 'hakkında',
 'hala',
 'haydi',
 'hemen',
 'her',
 'hey',
 'hiç',
 'hiçbir',
 'ile',
 'ilk',
 'istiyorsun',
 'iyi',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kadar',
 'kendi',
 'ki',
 'kimse',
 'küçük',
 'kız',
 'lazım',
 'merhaba',
 'mü',
 'mısın',
 'nasıl',
 'oh',
 'olabilir',
 'olacak',
 'olan',
 'olarak',
 'olsun',
 'onlar',
 'onları',
 'onu',
 'onun',
 'onunla',
 'orada',
 'para',
 'pekala',
 'saat',
 'sanırım',
 'selam',
 'seni',
 'siz',
 'sizi',
 'son',
 'sonra',
 'sorun',
 'tam',
 'tek',
 'tekrar',
 'teşekkür',
 'teşekkürler',
 'tüm',
 'uzun',
 'var',
 've',
 'ya',
 'yeni',
 'yine',
 'yoksa',
 'yüzden',
 'zaten',
 'çünkü',
 'önce',
 '

In [126]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [83]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [127]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_54_Threegram_Remain_Word_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [129]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [130]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [131]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [132]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

,not_in_youtube_result
0,hadi al şunu
1,şimdi olmaz tamam
2,birlikte olacak mısın
3,haydi gidelim buradan
4,orada olmak istiyorum
...,...
168,üç adam vardı
169,bu küçük adam
170,bu sorun değil
171,gün bugün mü


In [133]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

173

In [134]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [135]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [136]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2252.071,2254.696,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3324.000,3325.000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7783.760,7786.920,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1722.080,1731.530,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.315,8140.215,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1818.060,1819.340,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.804,1818.991,o gün bugün mü,KnZuRjwzbus


In [137]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

67

In [138]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2252.071000,2252.621403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3324.000000,3325.000000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7784.747500,7786.130000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1727.480000,1730.302727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.821667,8140.215000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1818.060000,1818.913333,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.888786,1818.991000,o gün bugün mü,KnZuRjwzbus


In [139]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3323.400000,3325.600000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7784.147500,7786.730000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1726.880000,1730.902727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.221667,8140.815000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1817.460000,1819.513333,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.288786,1819.591000,o gün bugün mü,KnZuRjwzbus


In [140]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id,video_url
0,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE,https://www.youtube.com/watch?v=lREyKzqpCjE&t=...
1,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY,https://www.youtube.com/watch?v=io3ajc3ysBY&t=...
2,hadi al şunu,3323.400000,3325.600000,hadi al şunu,oE9qrxR7DpI,https://www.youtube.com/watch?v=oE9qrxR7DpI&t=...
3,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw,https://www.youtube.com/watch?v=Ri8OHAHmTJw&t=...
4,hadi al şunu,7784.147500,7786.730000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc,https://www.youtube.com/watch?v=Q9bo4mKKszc&t=...
...,...,...,...,...,...,...
177,ne olduğunu biliyorum,1726.880000,1730.902727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY,https://www.youtube.com/watch?v=Tyx5100s0lY&t=...
178,ne olduğunu biliyorum,8138.221667,8140.815000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM,https://www.youtube.com/watch?v=8Kq9E5l0SPM&t=...
179,üç adam vardı,1817.460000,1819.513333,üç adam vardı yanında,e3rIly3dY5E,https://www.youtube.com/watch?v=e3rIly3dY5E&t=...
180,gün bugün mü,1817.288786,1819.591000,o gün bugün mü,KnZuRjwzbus,https://www.youtube.com/watch?v=KnZuRjwzbus&t=...


In [141]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

67

In [142]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 